In [10]:
import os
import json
import logging
import nest_asyncio
nest_asyncio.apply()
from dotenv import load_dotenv
from datetime import datetime
from pathlib import Path
from telethon import TelegramClient, events, errors
load_dotenv()

API_ID = os.getenv("API_ID")
API_HASH = os.getenv("API_HASH")

In [11]:
CHANNELS = [
    'https://t.me/lobelia4cosmetics',
    'https://t.me/tikvahpharma',
    'chemedtelegram', # Username for Chemed
]

# Path Setup
BASE_DATA_PATH = Path("data/raw")
IMAGE_BASE_PATH = BASE_DATA_PATH / "images"
JSON_BASE_PATH = BASE_DATA_PATH / "telegram_messages"
LOG_PATH = Path("logs")

In [12]:
for path in [IMAGE_BASE_PATH, JSON_BASE_PATH, LOG_PATH]:
    path.mkdir(parents=True, exist_ok=True)

# Logging Setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_PATH / "scraping.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [13]:
async def scrape_channel(client, channel_username):
    """Scrapes messages and images from a single channel."""
    try:
        # Clean channel name for folder usage
        entity = await client.get_entity(channel_username)
        channel_name = entity.username or entity.title
        logger.info(f"Starting scrape for: {channel_name}")

        messages_data = []
        
        async for message in client.iter_messages(entity, limit=100): # Limit set to 100 for testing
            # Prepare message metadata
            msg_id = message.id
            msg_date = message.date.strftime("%Y-%m-%d")
            
            # 1. Download Image if present
            image_path = None
            if message.photo:
                image_folder = IMAGE_BASE_PATH / channel_name
                image_folder.mkdir(parents=True, exist_ok=True)
                
                # Format: data/raw/images/{channel_name}/{message_id}.jpg
                file_name = f"{msg_id}.jpg"
                save_path = image_folder / file_name
                
                image_path = await client.download_media(message.photo, file=str(save_path))
                logger.info(f"Downloaded image for msg {msg_id}")

            # 2. Extract Data structure
            data = {
                "message_id": msg_id,
                "date": message.date.isoformat(),
                "text": message.text,
                "views": message.views,
                "forwards": message.forwards,
                "image_path": str(image_path) if image_path else None,
                "channel": channel_name
            }
            messages_data.append(data)

        # 3. Store in Data Lake (Partitioned by Date)
        today_str = datetime.now().strftime("%Y-%m-%d")
        partition_path = JSON_BASE_PATH / today_str
        partition_path.mkdir(parents=True, exist_ok=True)
        
        json_file = partition_path / f"{channel_name}.json"
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(messages_data, f, ensure_ascii=False, indent=4)
        
        logger.info(f"Successfully saved {len(messages_data)} messages for {channel_name}")

    except errors.FloodWaitError as e:
        logger.error(f"Rate limit exceeded. Wait for {e.seconds} seconds.")
        await asyncio.sleep(e.seconds)
    except Exception as e:
        logger.error(f"Error scraping {channel_username}: {str(e)}")
async def main():
    async with TelegramClient('scraper_session', API_ID, API_HASH) as client:
        for channel in CHANNELS:
            await scrape_channel(client, channel)

if __name__ == "__main__":
    asyncio.run(main())

2026-01-19 20:11:44,082 - INFO - Connecting to 149.154.167.51:443/TcpFull...
2026-01-19 20:11:46,923 - INFO - Connection to 149.154.167.51:443/TcpFull complete!
2026-01-19 20:12:54,419 - INFO - Phone migrated to 4
2026-01-19 20:12:54,537 - INFO - Reconnecting to new data center 4
2026-01-19 20:12:54,653 - INFO - Disconnecting from 149.154.167.51:443/TcpFull...
2026-01-19 20:12:54,657 - INFO - Disconnection from 149.154.167.51:443/TcpFull complete!
2026-01-19 20:12:54,659 - INFO - Connecting to 149.154.167.91:443/TcpFull...
2026-01-19 20:12:56,659 - INFO - Connection to 149.154.167.91:443/TcpFull complete!
2026-01-19 20:13:12,090 - INFO - Starting scrape for: lobelia4cosmetics


Signed in successfully as Zemicahel; remember to not break the ToS or you will risk an account ban!


2026-01-19 20:13:12,681 - INFO - Starting direct file download in chunks of 131072 at 0, stride 131072
2026-01-19 20:13:13,357 - INFO - Downloaded image for msg 22946
/Users/mac/.pyenv/versions/3.10.11/lib/python3.10/encodings/utf_16_le.py:25: RuntimeWarning: coroutine 'main' was never awaited
  class StreamWriter(codecs.StreamWriter):
2026-01-19 20:13:13,777 - INFO - Starting direct file download in chunks of 131072 at 0, stride 131072
2026-01-19 20:13:14,370 - INFO - Downloaded image for msg 22945
2026-01-19 20:13:14,373 - INFO - Starting direct file download in chunks of 131072 at 0, stride 131072
2026-01-19 20:13:15,133 - INFO - Downloaded image for msg 22944
2026-01-19 20:13:15,140 - INFO - Starting direct file download in chunks of 131072 at 0, stride 131072
2026-01-19 20:13:15,806 - INFO - Downloaded image for msg 22943
2026-01-19 20:13:15,808 - INFO - Starting direct file download in chunks of 131072 at 0, stride 131072
2026-01-19 20:13:16,278 - INFO - Downloaded image for msg 